<a href="https://colab.research.google.com/github/Weizhuo-Zhang/c-_primer_plus/blob/master/chapter08_function/chapter_08_function_notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 第8章 函数探幽

---

## 8.5 函数模板

函数模板是通用的函数描述，也就是说，它们使用泛型来定义函数，其中的泛型可用具体的类型（如int或double）替换。通过将类型作为参数传递给模板，可使编译器生成该类型的函数。由于模板允许以泛型（而不是具体类型）的方式编写程序。因此有时也被称为通用编程。由于类型是用参数表示的，因此模板特性有时也被称为参数化类型(parameterized types)。

函数模板允许以任意类型的方式来定义函数。例如，可以这样建立一个交换模板：
```c
template <typename AnyType>
void Swap(AnyType &a, AnyType &b)
{
    AnyType temp;
    temp = a;
    a = b;
    b = temp;
}
```

第一行指出，要建立一个模板，并将类型命名为`AnyType`。关键字template 和 typename 是必需的，除非可以使用关键字`class`代替`typename`。另外必须使用尖括号。类型名可以任意选择(这里为`AnyType`)


#### 程序清单 8.11 funtemp.cpp
```c
//funtemp.cpp -- using a function template
#include <iostream>
// Function template prototype
template <typename T>          // or class T
void Swap(T &a, T &b);

int main()
{
    using namespace std;
    int i = 10;
    int j = 20;
    cout << "i, j = " << i << ", " << j << ".\n";
    cout << "Using compiler generated int swapper:\n";
    Swap(i, j);    // generates void Swap(int &, int &)
    cout << "Now i, j = " << i << ", " << j << ".\n";
    
    double x = 24.5;
    double y = 81.5;
    cout << "x, y = " << x << ", " << y << ".\n";
    cout << "Using compiler generated double swapper:\n";
    Swap(x, y);    // generates void Swap(double &, double &)
    cout << "Now x, y = " << x << ", " << y << ".\n";
    return 0;
}

// function template definition
template <typename T> // or class T
void Swap(T &a, T &b)
{
    T temp;           // temp a variable of type T
    temp = a;
    a = b;
    b = temp;
}
```

注意，模板函数不能缩短可执行程序。对于程序清单8.11，最终仍由两个独立的函数定义。更常见的情形是，将模板放在头文件中，并在需要使用模板的文件中包含头文件。

### 8.5.1 重载的模板

需要多个对不同类型使用同一种算法的函数时，可以使用模板，如程序清单8.11所示。然而，并非所有的类型都使用相同的算法。为满足这种需求，可以像重载常规函数定义那样重载模板定义。和常规重载一样，被重载的模板的函数的特征标必须不同。例如，程序清单8.12新增了一个交换模板，用于交换两个数组中的元素。原来的模板的特征标为(T &, T &)，而新模板的特征标为(T [], T[], int)。注意，在后一个模板中，最后一个参数的类型为具体类型(int)。而不是泛型。并非所有的模板参数都必须是模板参数类型。

**程序清单8.12 twotemps.cpp**
```c
//twotemps.cpp -- using overloaded template funcitons
#include <iostream>
template <typename T>                // original template
void Swap(T &a, T &b);

template <typename T>
void Swap(T *a, T *b, int n);

void Show(int a[]);
const int Lim = 8;
int main()
{
    using namespace std;
    int i = 10, j = 20;
    cout << "i , j = " << i << ", " << j << ".\n";
    cout << "Using compiler-generated int swapper;\n";
    Swap(i, j);                      // matches original template
    cout << "Now i , j = " << i << ", " << j << ".\n";
    
    int d1[Lim] = {0, 7, 0, 4, 1, 7, 7, 6};
    int d2[Lim] = {0, 7, 2, 0, 1, 9, 6, 9};
    cout <<"Original arrays:\n";
    Show(d1);
    Show(d2);
    Swap(d1, d2, Lim);               // matches new template
    cout << "Swapped arrays:\n";
    Show(d1);
    Show(d2);    
    return 0;
}

template <typename T>
void Swap(T &a, T &b)
{
    T temp;
    temp = a;
    a = b;
    b = temp;
}

template <typename T>
void Swap(T a[], T b[], int n)
{
    T temp;
    for (int i = 0; i < n; i++)
    {
        temp = a[i];
        a[i] = b[i];
        b[i] = temp;
    }
}

void Show(int a[])
{
    using namespace std;
    cout << a[0] << a[1] << "/";
    cout << a[2] << a[3] << "/";
    for (int i = 4; i < Lim; i++)
        cout << a[i];
    cout << endl;
}
```

---

### 8.5.2 模板的局限性

假设有如下模板函数：
```c
template <class T>
void f(T a, T b)
{...}
```

通常，代码假定可执行哪些操作。例如，下面的代码假定定义了赋值，但如果T为数组，这种假设将不成立：
```c
a = b;
```

同样，下面的语句假设定义了`<`，但如果`T`为结构，该假设便不成立：
```c
if (a > b)
```

总之，编写的模板函数很可能无法处理某些类型。另一方面，有时候通用化是有意义的，但C++语法不允许这样做。例如，将两个包含位置坐标的结构相加是有意义的，虽然没有为结构定义运算符`+`。一种解决方案是，C++允许重载运算符`+`，以便能够将其用于特定的结构或类。另一种解决方案是，为特定类型提供具体化的模板定义。

### 8.5.3 显式具体化(explcit specialization)

```c
struct job
{
    char name[40];
    souble salary;
    int floor;
};
```
由于C++允许将一个结构赋给另一个结构，因此即使T是一个job结构，上述代码也适用。然而，假设只想交换salary 和 floor 成员，而不交换 name 成员，则需要使用不同的代码，但Swap()的参数将保持不变(两个job结构的引用)，因此无法使用模板重载来提供其他的代码。

#### 1. 第三代具体化(ISO/ANSI C++标准)
- 对于给定的函数名，可以有非模板函数，模板函数和显式具体化模板函数以及他们的重载版本。
- 显式具体化的原型和定义应以template<>打头，并痛过名称来指出类型。
- **优先级** 非模板函数 > 具体化模板函数 > 常规模板函数

下面是用于交换job结构的非模板函数、模板函数和具体化的原型：
```c
// nn template function prototype
void Swap(job &, job &);

//template prototype
template <typename T>
void Swap(T &, T &);

// explicit spcialization for the job type
template <> void Swap<job> (job &, job &);
```

在下面的代码中，第一次调用Swap()时使用通用版本，而第二次调用使用基于job类型的显式具体化版本。

```c
template <class T>              // template
void Swap(T &, T &);

// explicit specialization for the job type
template <> void Swap<job>(job &, job &);
int main()
{
    double u, v;
    Swap(u, v);         // use template
    job a,b;
    Swap(a, b);         // use void Swap<job>(job &, job &)
}
```

`Swap<job>`中的`<job>`是可选的，因为函数的参数类型表明，这是job的一个具体化。因此，该原型也可以这样写：
```c
template <> void Swap(job &, job &);      // simpler form
```

#### 2. 显式具体化示例
**程序清单8.13 twoswap.cpp**
```c
// twoswap.cpp -- specialization overrides a template
#include <iostream>
template <typename T>
void Swap(T &, T &);

struct job
{
    char name[40];
    double salary;
    int floor;
};

// explicit specialization
template <> void Swap<job>(job &j1, job &j2);
void Show(job &j);

int main()
{
    using namespace std;
    cout.precision(2);
    cout.setf(ios::fixed, ios::floatfield);
    int i = 10, j = 20;
    cout << "i, j = " << i << ", " << j << ".\n";
    cout << "using compiler generated int swapper:\n";
    Swap(i, j);    // generates void Swap(int &, int &)
    cout << "Now i, j = " << i << ", " << j << ".\n";
    
    job sue = {"Susan Yaffee", 73000.60, 7};
    job sidney{"Sidney Taffee", 78000.72, 9};
    cout << "Before job swapping:\n";
    Show(sue);
    Show(sidney);
    Swap(sue, sidney);       // uses void Swap(job &, job &)
    cout << "After job swapping:\n";
    Show(sue);
    Show(sidney);
    return 0;
}

template <typename T>
void Swap(T &a, T &b)
{
    T temp;
    temp = a;
    a = b;
    b = temp;
}

template <> void Swap<job>(job &j1, job &j2) // specialization
{
    double t1;
    int t2;
    t1 = j1.salary;
    j1.salary = j2.salary;
    j2.salary = t1;
    t2 = j1.floor;
    j1.floor = j2.floor;
    j2.floor = t2;
}

void Show(job &j)
{
    using namespace std;
    cout << j.name << ": $" << j.salary
         << " on floor " << j.floor << endl;
}
```

---

### 8.5.4 实例化和具体化

在代码中包含函数模板本身并不会生成函数定义，它只是一个用于生成函数定义的方案。编译器使用模板为特定类型生成函数定义时，得到的是模板实例(instantiation)。例如，在程序清单8.13中，函数调用`Swap(i, j)`导致编译器生成`Swap()`的一个实例，该实例使用`int`类型。模板并非函数定义，但使用`int`的模板实例是函数定义。这种实例化方式被称为隐式实例化(implicit instantiation)，因为编译器之所以知道需要进行定义，是由于程序调用`Swap()`函数时提供了int参数。

还可以在程序中使用函数来创建显式实例化：
```c
template <typename T>
T Add(T a, T b)
{
    return a + b;
}
...
int m = 6;
double x = 10.2;
cout << Add<double>(x, m) << endl;  // explicit instantiation
```

这里的模板与函数调用 `Add(x, m)` 不匹配，因为改模板要求两个函数参数的类型相同。但通过使用 `Add<double>(x, m)`，可强制为 double 类型实例化。并将参数 m 强制转换为 double 类型，以便于函数 `Add<double>(double, double)`的第二个参数匹配。

---

### 8.5.5 编译器选择使用那个函数版本

**重载解析(Overloading resolution)**：C++用于决定为函数调用使用哪一个函数定义。

**步骤**

1. 创建候选函数列表，其中包含与被调用函数名称相同的函数和模版函数。
2. 使用候选函数列表创建可行函数列表。这些都是参数数目正确的函数，为此有一个隐式转换序列，其中包括实参类型与相应的形参类型完全匹配的情况。例如，使用float参数的函数调用可以将参数转为double。
3. 确定是否有最佳的可行函数。如果有，则使用它，否则该函数调用出错。

可行的候选函数，最佳到最差的顺序：
1. 完全匹配，但常规函数优先于模版函数。
2. 提升转换(例如，char和shorts自动转为int，float自动转为double)。
3. 标准转换(例如，int转为char，long转为double)。
4. 用户定义的转换，如类声明定义的转换。

#### 1. 完全匹配和最佳匹配
如果有多个匹配的原型，编译器将无法完成重载解析过程。

#### 2. 部分排序规则

---

### 8.5.6 模版函数的发展

#### 2. 关键字`decltype`(C++11)
```c
int x;
decltype(x) y;      // make y the same as x
decltype(x+y) xpy   // make xpy the same type as x+y
```